In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [18]:
class Net(nn.Module):

    def __init__(self):
        super(Net,self).__init__()

        # 输入图像channel:1,输出channel：6，5x5卷积
        self.conv1 = nn.Conv2d(1,6,5)
        # 1,16,5*5
        self.conv2 = nn.Conv2d(6,16,5)

        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        # 最终输出节点10个
        self.fc3 = nn.Linear(84,10)
    
    def forward(self,x):
        # 2*2最大池化
        # 对卷积的输出relu,然后最大池化
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        # 如果是方阵，也可以只用一个数字定义
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = x.view(-1,self.num_flat_features(x))

        # 全连接层后relu
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self,x):
        # 除去批处理维度的其他所有维度
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [19]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


In [20]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[-0.0598,  0.0428, -0.0834,  0.0769,  0.0159,  0.0764,  0.0542, -0.1249,
         -0.0289, -0.0186]], grad_fn=<AddmmBackward>)


In [21]:
net.zero_grad()
out.backward(torch.randn(1,10))

In [22]:
output = net(input)
target = torch.randn(1,10)
target = target.view(1,-1)
criterion = nn.MSELoss()

loss = criterion(output,target)
print(loss)

tensor(1.3487, grad_fn=<MseLossBackward>)


In [23]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0]) # linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) #relu

In [24]:
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0100, -0.0025, -0.0024,  0.0232, -0.0103, -0.0138])


In [25]:
# learning_rate = 0.01
# for f in net.parameters():
#     f.data.sub_(f.grad.data * learning_rate)

import torch.optim as optim

# 创建优化器(optimizer）
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 在训练的迭代中：
optimizer.zero_grad()   # 清零梯度缓存
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # 更新参数